In [1]:
# dd

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn as sk
import math
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [3]:
CONFIG = {
    'path_train' : '/kaggle/input/titanic/train.csv',
    'path_test' : '/kaggle/input/titanic/test.csv',
    'path_submission' : '/kaggle/input/titanic/gender_submission.csv',
}

In [4]:
df_train_raw = pd.read_csv(CONFIG['path_train'])
df_test_raw = pd.read_csv(CONFIG['path_test'])
df_submission_raw = pd.read_csv(CONFIG['path_submission'])

df_train_raw.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
def is_str_list(values):
    return any([type(v) is str for v in values])
def encode(df:pd.DataFrame, feature:str, keys):
    df[feature] = [keys.index(v) if type(v) is str else v for v in df[feature].values]
def refine_df(df:pd.DataFrame, features, na_value:int=0):
    df = df[features].fillna(na_value)
    for feature in features:
        keys = list(set(df[feature].values))
        if not is_str_list(keys): continue
        if na_value in keys: keys.remove(na_value)
        encode(df, feature, keys)
        print(f'{feature}{keys} has been encoded.')
    return df

In [6]:
features = [
'Pclass',
'Sex',
'Age',
'SibSp',
'Parch',
'Fare',
'Embarked',
]

In [7]:
X = refine_df(df_train_raw, features)
y = df_train_raw['Survived']

Sex['male', 'female'] has been encoded.
Embarked['C', 'Q', 'S'] has been encoded.


In [8]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 21)
test_X = refine_df(df_test_raw, features)

Sex['male', 'female'] has been encoded.
Embarked['C', 'Q', 'S'] has been encoded.


In [9]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
model = XGBClassifier(random_state=21)#
model.fit(train_X, train_y)
# train_predictions = model.predict(train_X)
# val_predictions = model.predict(val_X)

train_score = model.score(train_X, train_y)
val_score = model.score(val_X, val_y)
print(f"Train Score : {train_score} / Valid Score : {val_score}")

Train Score : 0.9745508982035929 / Valid Score : 0.8251121076233184


In [10]:
df_submission = pd.DataFrame(df_submission_raw.to_dict())
df_submission['Survived'] = model.predict(test_X)
df_submission.to_csv('submission.csv', index=False)